In [ ]:
# prompt: google drive mount

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

# 데이터 경로 설정
train_data_path = "/content/drive/MyDrive/AIR/aug"
test_data_path = "/content/drive/MyDrive/AIR/test"

# 선택한 3개의 클래스
selected_classes = ['0', '1', '2']

# 데이터 전처리 및 증강
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# 데이터 로더 생성
train_dataset = datasets.ImageFolder(root=train_data_path, transform=transform)

# Filter out data for only the selected classes
train_dataset.samples = [
    (image, label) for image, label in train_dataset.samples if train_dataset.classes[label] in selected_classes
]
train_dataset.targets = [label for _, label in train_dataset.samples]

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

test_dataset = datasets.ImageFolder(root=test_data_path, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# ResNet 모델 불러오기
model = models.resnet18(pretrained=True)
# 마지막 Fully Connected Layer를 선택한 3개의 클래스에 맞게 조정
model.fc = nn.Linear(model.fc.in_features, len(selected_classes))

# 손실 함수 및 최적화 함수 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 학습
num_epochs = 17
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    if epoch>=5:
       torch.save(model.state_dict(), f"/content/drive/MyDrive/AIR/resnet_pre_epoch_{epoch}.pth")
       # 테스트
       model.eval()
       correct = 0
       total = 0

       with torch.no_grad():
             for inputs, labels in test_loader:
                 inputs, labels = inputs.to(device), labels.to(device)
                 outputs = model(inputs)
                 _, predicted = torch.max(outputs.data, 1)
                 total += labels.size(0)
                 correct += (predicted == labels).sum().item()

             print(f"Accuracy on test set: {100 * correct / total}%")


    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

#torch.save(model.state_dict(), "/content/drive/MyDrive/AIR/resnet_model_selected_classes.pth")

# 테스트
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total}%")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/17, Loss: 0.8437826603651046
Epoch 2/17, Loss: 0.26745012588799
Epoch 3/17, Loss: 0.20186726885537307
Epoch 4/17, Loss: 0.09262221672882637
Epoch 5/17, Loss: 0.09571815395417313
Accuracy on test set: 100.0%
Epoch 6/17, Loss: 0.08501489725507175
Accuracy on test set: 100.0%
Epoch 7/17, Loss: 0.033041211970460914
Accuracy on test set: 100.0%
Epoch 8/17, Loss: 0.03452343909302726
Accuracy on test set: 100.0%
Epoch 9/17, Loss: 0.0924679522557805
Accuracy on test set: 100.0%
Epoch 10/17, Loss: 0.01700935799550886
Accuracy on test set: 100.0%
Epoch 11/17, Loss: 0.026282797466653088
Accuracy on test set: 100.0%
Epoch 12/17, Loss: 0.046500966225479105
Accuracy on test set: 100.0%
Epoch 13/17, Loss: 0.02938907714560628
Accuracy on test set: 100.0%
Epoch 14/17, Loss: 0.022159177217205676
Accuracy on test set: 100.0%
Epoch 15/17, Loss: 0.02068830677793206
Accuracy on test set: 100.0%
Epoch 16/17, Loss: 0.022364748428420473
Accuracy on test set: 100.0%
Epoch 17/17, Loss: 0.0147186381043866

In [ ]:

# 모델 인스턴스 생성
model = CustomResNet18(num_classes=3)

# 학습된 가중치 로드
state_dict = torch.load("/content/drive/MyDrive/AIR/resnet__nonpre_epoch_17.pth")
# 모델의 state_dict와 불러온 state_dict의 키를 비교해 일치하도록 수정
model_dict = model.state_dict()
state_dict = {k: v for k, v in state_dict.items() if k in model_dict}
model_dict.update(state_dict)
model.load_state_dict(model_dict)
# 테스트
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total}%")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/AIR/resnet__nonpre_epoch_17.pth'

In [ ]:
import torch
from torchvision import transforms, models
from PIL import Image
import time

# 모델 정의
class CustomResNet18(nn.Module):
    def __init__(self, num_classes=3):
        super(CustomResNet18, self).__init__()
        self.resnet18 = models.resnet18(pretrained=False)
        in_features = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.resnet18(x)

# 모델 인스턴스 생성
model = CustomResNet18(num_classes=3)

# 학습된 가중치 로드
state_dict = torch.load("/content/drive/MyDrive/AIR/resnet_model_selected_classes.pth")
# 모델의 state_dict와 불러온 state_dict의 키를 비교해 일치하도록 수정
model_dict = model.state_dict()
state_dict = {k: v for k, v in state_dict.items() if k in model_dict}
model_dict.update(state_dict)
model.load_state_dict(model_dict)
model.eval()


# 이미지 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# 분류할 이미지 경로
image_path = "/content/drive/MyDrive/AIR/test/1/wing1.jpg"

# 이미지 불러오기 및 전처리
image = Image.open(image_path)
input_tensor = transform(image)
input_batch = input_tensor.unsqueeze(0)  # 배치 차원 추가

start_time = time.time()
# 모델에 입력 전달 및 예측
with torch.no_grad():
    output = model(input_batch)

# 예측 결과 확인
_, predicted_class = torch.max(output, 1)
end_time = time.time()
execution_time = end_time - start_time
print("Predicted class:", predicted_class.item())
print(f"Execution Time: {execution_time} seconds")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Predicted class: 0
Execution Time: 0.09061145782470703 seconds
